In [62]:
import chromadb
import fitz
import spacy
from sentence_transformers import SentenceTransformer
from pprint import pprint
from pathlib import Path
import os
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
import openai

In [63]:
#Listando arquivos da pasta local "pdf_documents"

pdf_folder_path = "../pdf_documents"
pdf_files = [f for f in os.listdir(pdf_folder_path) if f.endswith(".pdf")]

print("Arquivos encontrados:")
for pdf in pdf_files:
    print(f"- {pdf}")

Arquivos encontrados:
- EIA_example1.pdf
- RIMA_example1.pdf


In [64]:
#função para extrair o texto do arquivo pdf


#import spacy
import fitz
import re

nlp = spacy.load("pt_core_news_sm")

IGNORE_KEYWORDS = {
    "sumário", "índice", "resumo", "anexo", "figura", "tabela", "referência", "bibliografia",
    "conteúdo", "protocolo", "gov", "secretaria", "manifestação",
    "certidão", "anuência", "processo", "cbh", "condphaat", "comitê", 
    "autenticação mecânica", "assinatura"

}

def extract_relevant_sentences(text, min_words=3, require_verb=False):
    doc = nlp(text)
    relevant = []

    for sent in doc.sents:
        text = sent.text.strip()
        if len(text.split()) < min_words:
            continue
        if require_verb and not any(tok.pos_ == "VERB" for tok in sent):
            continue
        relevant.append(text)
    return relevant

def is_line_irrelevant(line):
    lower = line.lower()

    if len(line) < 30:
        return True
    if any(k in lower for k in IGNORE_KEYWORDS):
        return True
    if lower.endswith(".pdf"):
        return True
    if re.search(r"\.{3,}\s*\d{1,3}$", line):
        return True
    if re.match(r"^\d+(\.\d+)+\s+", line):
        return True
    if re.match(r"^(Mapa|GEO)\s*-\s*", line):
        return True
    if line.isupper() and len(line.split()) > 4:
        return True
    if re.search(r"\d{8,}", line):
        return True
    if re.fullmatch(r"_+", line):
        return True
    if re.search(r"\d{2}/\d{2}/\d{2,4}", line):
        return True
    if re.search(r"[A-Fa-f0-9]{10,}", line):
        return True

    return False

def extract_clean_text_with_spacy(pdf_path):
    try:
        with fitz.open(pdf_path) as doc:
            raw_text = "\n".join([page.get_text("text") for page in doc])

        lines = raw_text.splitlines()
        filtered = [line.strip() for line in lines if not is_line_irrelevant(line.strip())]
        cleaned_text = " ".join(filtered)

        relevant_sentences = extract_relevant_sentences(cleaned_text, min_words=4, require_verb=False)
        return "\n".join(relevant_sentences) if relevant_sentences else None

    except Exception as e:
        print(f"Erro ao processar {pdf_path}: {e}")
        return None




In [65]:
# Exemplo de funcionamento da função com o primeiro arquivo da pasta
if pdf_files:
    pdf_path = os.path.join(pdf_folder_path, pdf_files[0]) 
    pdf_text = extract_clean_text_with_spacy(pdf_path)  
    print("Exemplo de texto extraído do primeiro arquivo da pasta")
    print(f"\n Nome do arquivo : {pdf_files[0]}\n")
    print(pdf_text)
else:
    print("Lista de pdf_files vazia")


Exemplo de texto extraído do primeiro arquivo da pasta

 Nome do arquivo : EIA_example1.pdf

Unidade de Recuperação de Energia INCLUIR DOCUMENTOS SOLICITADOS
Para cada documento a ser anexado, gerar um único arquivo digital, com no máximo 50MB.
Os arquivos acima de 50MB (estudos ambientais) deverão ser particionados para atender o limite, de acordo com a divisão indicada neste Formulário.
Antes da inclusão dos arquivos recomenda-se assistir ao vídeo explicativo, ler a sessão de “Perguntas Frequentes” e verificar a licenciamento com avaliação de impacto ambiental no sistema eletrônico e-ambiente”, explicitando principalmente instruções sobre organização, partição, qualidade e formato dos documentos.
Se identificadas situações em desacordo com a Decisão de Diretoria, o interessado será notificado, por e-mail, a fazer novo
Este Formulário contém 12 páginas.
Preencher somente os campos pertinentes à solicitação desejada.
Não inserir o mesmo documento em mais de um campo do Formulário.
Pági

In [66]:
#Extraindo o texto de todos os arquivos da pasta

pdf_texts = {}

for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_folder_path, pdf_file)

    print(f"Processando: {pdf_file}...")  # Mostra qual arquivo está sendo lido

    try:
        pdf_texts[pdf_file] = extract_clean_text_with_spacy(pdf_path)  
        print(f"Extração concluída: {pdf_file}\n")
    except Exception as e:
        print(f"Erro ao processar {pdf_file}: {e}\n")

print(f"\nOs textos de {len(pdf_texts)} documentos foram extraídos com sucesso.")


Processando: EIA_example1.pdf...
Extração concluída: EIA_example1.pdf

Processando: RIMA_example1.pdf...
Extração concluída: RIMA_example1.pdf


Os textos de 2 documentos foram extraídos com sucesso.


In [67]:
#O erro "MuPDF error: library error: zlib error: invalid distance too far back" geralmente indica que há uma página em branco no documento,
#mas não impede a extração do texto, permitindo a continuação do processamento


In [68]:
# Carregar a chave da API do ambiente
openai_api_key = os.getenv("OPENAI_API_KEY")

In [69]:

#Função para dividir os textos em chunks
def split_text_into_chunks(text, chunk_size=512, chunk_overlap=50):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
    )
    
    chunks = splitter.split_text(text)
    return chunks

In [70]:
#Função para gerar os embeddings de cada chunk de acordo com a formtação da OPENAI
def generate_embeddings(chunks):
    embeddings_model = OpenAIEmbeddings(openai_api_key=openai_api_key)
    return embeddings_model.embed_documents(chunks)

In [71]:
#Gerando os chunks e embeedings para os textos extraídos dos documentos
text_chunks = {}
embeddings = {}

for pdf_file, full_text in pdf_texts.items():
    if not isinstance(full_text, str):  
        print(f"Aviso: O texto extraído de {pdf_file} não é uma string. Convertendo para string vazia.")
        full_text = ""
    
    text_chunks[pdf_file] = split_text_into_chunks(full_text)
    embeddings[pdf_file] = generate_embeddings(text_chunks[pdf_file])
    print(f"Texto do arquivo {pdf_file} dividido em {len(text_chunks[pdf_file])} blocos e embeddings gerados.")

print(f"\nOs textos de {len(pdf_texts)} documentos foram particionados e processados em embeddings.")


Texto do arquivo EIA_example1.pdf dividido em 592 blocos e embeddings gerados.
Texto do arquivo RIMA_example1.pdf dividido em 63 blocos e embeddings gerados.

Os textos de 2 documentos foram particionados e processados em embeddings.


In [72]:
print(type(embeddings))



<class 'dict'>


In [73]:
print(f"Nome dos arquivos processados para retirar os textos: {pdf_texts.keys()}")

Nome dos arquivos processados para retirar os textos: dict_keys(['EIA_example1.pdf', 'RIMA_example1.pdf'])


In [74]:
# Configurando o ChromaDB com persistência
chroma_client = chromadb.PersistentClient(path="./chroma_db")

In [75]:
chroma_client.delete_collection(name="pdf_documents")


In [76]:
# Criando a coleção 
collection = chroma_client.get_or_create_collection(
    name="pdf_documents",
    metadata={"hnsw:space": "cosine"},  # Algoritmo de similaridade
    embedding_function=None  # Permite adicionar embeddings manualmente
)


In [77]:

# Adicionando os embeddings ao ChromaDB
doc_id = 1  # Contador único para IDs
total_chunks = 0

for pdf_file, pdf_embeddings in embeddings.items():
    print(f"Salvando documento: {pdf_file} ({len(pdf_embeddings)} chunks)")

    for idx, embedding in enumerate(pdf_embeddings):
        chunk_id = f"{pdf_file}_chunk_{idx}"  # ID único para cada chunk
        collection.add(
            ids=[chunk_id],
            embeddings=[embedding],
            metadatas=[{
                "chunk_index": idx,
                "numeric_id": doc_id,
                "source_file": pdf_file
            }]
        )
        doc_id += 1
        total_chunks += 1

    print(f" -> Documento '{pdf_file}' salvo com sucesso.\n")

print(f"\n{total_chunks} chunks armazenados no ChromaDB.")
print("Todos os documentos foram salvos no banco ChromaDB.")


Salvando documento: EIA_example1.pdf (592 chunks)
 -> Documento 'EIA_example1.pdf' salvo com sucesso.

Salvando documento: RIMA_example1.pdf (63 chunks)
 -> Documento 'RIMA_example1.pdf' salvo com sucesso.


655 chunks armazenados no ChromaDB.
Todos os documentos foram salvos no banco ChromaDB.


In [78]:
print("ids dos documentos:\n")
print(collection.get()["ids"])

ids dos documentos:

['EIA_example1.pdf_chunk_0', 'EIA_example1.pdf_chunk_1', 'EIA_example1.pdf_chunk_2', 'EIA_example1.pdf_chunk_3', 'EIA_example1.pdf_chunk_4', 'EIA_example1.pdf_chunk_5', 'EIA_example1.pdf_chunk_6', 'EIA_example1.pdf_chunk_7', 'EIA_example1.pdf_chunk_8', 'EIA_example1.pdf_chunk_9', 'EIA_example1.pdf_chunk_10', 'EIA_example1.pdf_chunk_11', 'EIA_example1.pdf_chunk_12', 'EIA_example1.pdf_chunk_13', 'EIA_example1.pdf_chunk_14', 'EIA_example1.pdf_chunk_15', 'EIA_example1.pdf_chunk_16', 'EIA_example1.pdf_chunk_17', 'EIA_example1.pdf_chunk_18', 'EIA_example1.pdf_chunk_19', 'EIA_example1.pdf_chunk_20', 'EIA_example1.pdf_chunk_21', 'EIA_example1.pdf_chunk_22', 'EIA_example1.pdf_chunk_23', 'EIA_example1.pdf_chunk_24', 'EIA_example1.pdf_chunk_25', 'EIA_example1.pdf_chunk_26', 'EIA_example1.pdf_chunk_27', 'EIA_example1.pdf_chunk_28', 'EIA_example1.pdf_chunk_29', 'EIA_example1.pdf_chunk_30', 'EIA_example1.pdf_chunk_31', 'EIA_example1.pdf_chunk_32', 'EIA_example1.pdf_chunk_33', 'E

In [79]:
#exemplo de consulta de documento por id numérico nos metadados

result = collection.get(where={"numeric_id": 5})  # Busca pelo ID numérico armazenado como metadado

print(result)


{'ids': ['EIA_example1.pdf_chunk_4'], 'embeddings': None, 'documents': [None], 'uris': None, 'included': ['metadatas', 'documents'], 'data': None, 'metadatas': [{'numeric_id': 5, 'source_file': 'EIA_example1.pdf', 'chunk_index': 4}]}


In [89]:


# 🔹 Criando novamente o gerador de embeddings
openai_embeddings = OpenAIEmbeddings()  # Garante que usa o mesmo modelo dos chunks

# 🔹 Definindo a query de busca
query = "Qual a melhor forma de descartar óleo PCB?"
query_embedding = openai_embeddings.embed_query(query)  # Gera embedding para a query

# 🔹 Definindo o número de resultados desejados
n = 10

# 🔹 Buscando os N documentos mais relevantes no ChromaDB
results = collection.query(
    query_embeddings=[query_embedding],  
    n_results=n  
)

# 🔹 Exibindo os resultados
print("\n**Resultados da Busca:**")
for i, doc in enumerate(results["metadatas"][0]):
    print(f"{i+1}. Chunk {doc['chunk_index']} (Score: {results['distances'][0][i]})")



**Resultados da Busca:**
1. Chunk 74 (Score: 0.12672364711761475)
2. Chunk 293 (Score: 0.13301753997802734)
3. Chunk 1 (Score: 0.13471853733062744)
4. Chunk 9 (Score: 0.13614988327026367)
5. Chunk 60 (Score: 0.1388527750968933)
6. Chunk 32 (Score: 0.1391080617904663)
7. Chunk 290 (Score: 0.1406611204147339)
8. Chunk 292 (Score: 0.1410691738128662)
9. Chunk 92 (Score: 0.14127618074417114)
10. Chunk 301 (Score: 0.14200818538665771)


In [81]:
# Exibir os primeiros metadados para conferir os índices armazenados
results = collection.get()
print(results["metadatas"][:10])  # Mostra os 10 primeiros registros


[{'chunk_index': 0, 'numeric_id': 1, 'source_file': 'EIA_example1.pdf'}, {'chunk_index': 1, 'numeric_id': 2, 'source_file': 'EIA_example1.pdf'}, {'source_file': 'EIA_example1.pdf', 'chunk_index': 2, 'numeric_id': 3}, {'numeric_id': 4, 'chunk_index': 3, 'source_file': 'EIA_example1.pdf'}, {'chunk_index': 4, 'source_file': 'EIA_example1.pdf', 'numeric_id': 5}, {'chunk_index': 5, 'source_file': 'EIA_example1.pdf', 'numeric_id': 6}, {'source_file': 'EIA_example1.pdf', 'numeric_id': 7, 'chunk_index': 6}, {'numeric_id': 8, 'chunk_index': 7, 'source_file': 'EIA_example1.pdf'}, {'chunk_index': 8, 'numeric_id': 9, 'source_file': 'EIA_example1.pdf'}, {'numeric_id': 10, 'chunk_index': 9, 'source_file': 'EIA_example1.pdf'}]


In [90]:
def get_chunk_text_by_index(chunk_index):
    results = collection.get(where={"chunk_index": chunk_index})

    if results and "metadatas" in results and results["metadatas"]:
        chunk_id = results["ids"][0]  # Obtém o ID do chunk

        # Buscar o texto diretamente nos chunks armazenados
        for chunks in text_chunks.values():  
            if chunk_index < len(chunks):  
                return chunks[chunk_index]  # Retorna o texto do chunk

    return "Nenhum chunk encontrado com esse índice."

# Teste a função
chunk_index = 74  # Escolha o índice do chunk que deseja visualizar
chunk_text = get_chunk_text_by_index(chunk_index)

print(f"Texto do chunk {chunk_index}:\n{chunk_text}")


Texto do chunk 74:
A atividade vem ao encontro das necessidades de alternativas de tratamento e recuperação de óleos contaminados por As PCBs nunca foram fabricadas no Brasil, mas estima-se que houve uma importação de 14 a 26 mil toneladas de fluido PCB, o que corresponde a 1 a 2% das PCBs produzidas globalmente (BRASIL, 2013, BREIVIK et al, 2002 e BREIVIK et al, 2007).
Há muitas jurisdições e regulamentações rigorosas sobre o manuseio e descarte de materiais contaminados.


In [87]:



openai_embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

def search_query(query: str, n=5):
    query_embedding = openai_embeddings.embed_query(query)
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n
    )
    documents = results["documents"][0]

    documentos_filtrados = [doc for doc in documents if doc is not None]

    return "\n\n".join(documentos_filtrados)

#Função do chatbot com RAG
def chatbot(message: str):
    contexto = search_query(message)

    prompt = f"""
Você é um assistente especialista em licenciamento ambiental, com foco em auxiliar na elaboração de documentos EIA e RIMA. Use exclusivamente o contexto a seguir para responder de forma precisa e técnica.

Contexto extraído de documentos:
{contexto}

Pergunta do usuário:
{message}
"""
    client = openai.OpenAI(api_key=openai_api_key)
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "Você é um assistente técnico ambiental especializado em EIA e RIMA."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2,
        max_tokens=1000
    )
    return response.choices[0].message.content

# Loop de interação
while True:
    user_input = input("Você: ")
    if user_input.lower() in ["sair", "exit", "quit"]:
        break
    print("Bot:", chatbot(user_input))

Você:  Qual a melhor forma de descartar óleo PCB?


Bot: O descarte de óleo PCB (Bifenilas Policloradas) deve ser feito de maneira muito cuidadosa devido à sua alta toxicidade e persistência no ambiente. A melhor forma de descartar o óleo PCB é através da incineração em instalações apropriadas e licenciadas para tal. A incineração deve ser realizada a temperaturas superiores a 1200°C para garantir a destruição completa dos PCBs. 

É importante ressaltar que o manuseio e transporte do óleo PCB até a instalação de incineração deve ser feito por profissionais treinados e com equipamentos de proteção individual adequados. Além disso, todas as atividades devem estar em conformidade com as regulamentações locais e nacionais sobre o manuseio e descarte de substâncias perigosas.

Vale lembrar que o descarte inadequado de PCBs é ilegal e pode causar danos graves ao meio ambiente e à saúde humana. Portanto, é essencial que todas as precauções sejam tomadas para garantir um descarte seguro e eficaz.


Você:  SAIR


In [84]:
# parar de usar banco de dados em memória (talvez mudar para o postGre ou mongoDb) ou descobrir um jeito de salvar o chorma direito
# utilizar langgraph
#pesquisar sobre multiagentes (react) 
#Procurar forma de limpar documentos
#estudar leitura de imagens (leitura de mapas)
